In [3]:
# if you run into an error you may need to run this twice
!pip install boto3
!pip install aws-kinesis-agg
!pip install s3fs

  Using cached botocore-1.34.144-py3-none-any.whl (12.4 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.131
    Uninstalling botocore-1.34.131:
      Successfully uninstalled botocore-1.34.131
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.13.1 requires botocore<1.34.132,>=1.34.70, but you have botocore 1.34.144 which is incompatible.
  Using cached botocore-1.34.131-py3-none-any.whl (12.3 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.144
    Uninstalling botocore-1.34.144:
      Successfully uninstalled botocore-1.34.144
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
boto3 1.34.144 requires botocore<1.35.0,>=1.34.144, but you have botocore 1.34.13

**Make sure to create an `aws.cfg` file and upload it here**

aws.cfg example:

```
[AWS]
aws_access_key_id = youraccesskey
aws_secret_access_key = yoursecretkey
region_name=us-east-1
```
Make sure to change the values to match yours.

In [4]:
import boto3
import json
import time
import pandas as pd
import configparser

# Read AWS credentials from config file
config = configparser.ConfigParser()
config.read('aws.cfg')

aws_access_key_id = config['AWS']['aws_access_key_id']
aws_secret_access_key = config['AWS']['aws_secret_access_key']
region_name = config['AWS']['region_name']




In [7]:
# Initialize the boto3 client with credentials from config file
kinesis_client = boto3.client(
    'kinesis',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name
)


In [8]:
def produce(stream_name, data, partition_key):
    try:
        # Convert timestamps to strings
        for key in data:
            if isinstance(data[key], pd.Timestamp):
                data[key] = data[key].isoformat()
                # using the put_record method to push the stream
        response = kinesis_client.put_record(
            StreamName=stream_name,
            Data=json.dumps(data),
            PartitionKey=partition_key
        )
        return response
    except Exception as e:
        print(f"Error producing record: {e}")

# this function takes 5 records at a time and streams every 2-seconds
def stream_data(df, stream_name):
    for i in range(0, len(df), 5):
        records = df.iloc[i:i+5].to_dict(orient='records')
        for record in records:
            partition_key = str(record['tpep_pickup_datetime'])  # Use a valid column as the partition key

            # call the produce function
            produce(stream_name, record, partition_key)
        print(f"Sent {len(records)} records to Kinesis")
        time.sleep(2)


In [10]:
if __name__ == "__main__":
    stream_name = 'input-stream'
    # Reading the yellow_taxis parquet file and stream it
    df = pd.read_parquet('s3://techcatalyst-public/yellow_tripdata_2024-01.parquet',
                     storage_options={
                      'key': aws_access_key_id,
                     'secret': aws_secret_access_key,
                     })
    stream_data(df, stream_name)